## Cross-country analytics: Higher Education Tuition and Economic Indicators

    Creating BigQuery tables for dashboards


In [ ]:
# Load the required libraries
import pandas as pd
from google.cloud import bigquery
import os

In [ ]:
# Upload the json file with an API key to the Google Colab directory to access the BigQuery project
json_key_file = "json_key_file.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = json_key_file

a. Upload csv files to the BigQuery project

In [ ]:
# Function to upload CSV files to the BQ project

def upload_csv_to_BQ(file_path, table_id, schema=None):
    """
    Parameters:
    file_path: path to CSV file with data
    table_id: full table ID in BQ (project_id.dataset_id.table_id)
    schema: a table schema in needed (bigquery.SchemaField)
    Results:
    loaded to the BQ project csv file
    """

    client = bigquery.Client()

    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        autodetect=True if schema is None else False,
        skip_leading_rows=1,
        schema=schema,
        write_disposition=bigquery.WriteDisposition.WRITE_EMPTY,  # Error if this tabel already exists in BQ (WRITE_APPEND: append. WRITE_TRUNCATE: overwrite)
        field_delimiter=";",
        column_name_character_map='V2'  # Turn on Character Map V2
    )

    with open(file_path, "rb") as file:
        load_job = client.load_table_from_file(file, table_id, job_config=job_config)

    load_job.result()

    table = client.get_table(table_id)
    print(f"Loaded {table.num_rows} rows in the tabel {table_id}.")

In [ ]:
# Function to create a table schema with many consecutive years

def generate_schema(start_year, end_year, first_columns=None):
    schema = []

    if first_columns:
        schema.extend(first_columns)

    for year in range(start_year, end_year + 1):
        schema.append(bigquery.SchemaField(str(year), "FLOAT"))

    return schema

In [ ]:
client = bigquery.Client()

# Upload annual_tuition_fees.csv and WEOOct2024.csv

upload_csv_to_BQ(file_path='annual_tuition_fees.csv', table_id='zkv-work-with-cc-pubdatasets.HighEd_Market_Indicators.annual_tuition_fees')
upload_csv_to_BQ(file_path='WEOOct2024.csv', table_id='zkv-work-with-cc-pubdatasets.HighEd_Market_Indicators.WEOOct2024')

# Generate a schema and upload OECD_NAAG_Houshold_Income_new.csv
first_columns = [bigquery.SchemaField("Country", "STRING")]
start_year = 1970
end_year = 2023
schema = generate_schema(start_year, end_year, first_columns=first_columns)
upload_csv_to_BQ(file_path='OECD_NAAG_Houshold_Income_new.csv',
                 table_id='zkv-work-with-cc-pubdatasets.HighEd_Market_Indicators.OECD_NAAG_Houshold_Income',
                 schema=schema)

In [ ]:
# Check list of datasets and tables in the BigQuery project

datasets = list(client.list_datasets())
if datasets:
    print("Datasets in the project:")
    for dataset in datasets:
        print(f"{dataset.dataset_id}")

        tables = list(client.list_tables(dataset.dataset_id))
        if tables:
          print(f" Tables in the dataset:")
          for table in tables:
              print(f"  - {table.table_id}")
        else:
          print(" No tables in this dataset")
else:
    print("No datasets in the project")

b) Transforming tables in BigQuery from wide to long

In [ ]:
# Query to transform and create long type table OECD_NAAG_Houshold_Income for 3 years

query_long_IncomeTab = """
CREATE OR REPLACE TABLE `zkv-work-with-cc-pubdatasets.HighEd_Market_Indicators.Long_OECD_NAAG_Houshold_Income` AS
SELECT
  Country,
  Year,
  HH_Income
FROM
  `zkv-work-with-cc-pubdatasets.HighEd_Market_Indicators.OECD_NAAG_Houshold_Income`,
  UNNEST([
    STRUCT(2017 AS Year, `2017` AS HH_Income),
    STRUCT(2019 AS Year, `2019` AS HH_Income),
    STRUCT(2022 AS Year, `2022` AS HH_Income)
  ]) AS income_data
"""

client.query(query_long_IncomeTab)

In [ ]:
# Query to transform and create long type table LongSample_WEOOct2024 table for lists of countries, subjects and for 3 years
# Country: a list from annual_tuition_fees
# Years: 2017, 2019, 2022
# WEO Subject Code:
#  PPPGDP - Gross domestic product, current prices. Purchasing power parity; international dollars
#  PPPPC - Gross domestic product per capita, current prices. Purchasing power parity; international dollars
#  NID_NGDP - Total investment. Percent of GDP
#  LUR - Unemployment rate. Percent of total labor force

query_long_weo = """

CREATE OR REPLACE TABLE `zkv-work-with-cc-pubdatasets.HighEd_Market_Indicators.LongSample_WEOOct2024` AS

WITH

-- Temp table 1 for GDP
gdp_data AS(
  SELECT *
  FROM `zkv-work-with-cc-pubdatasets.HighEd_Market_Indicators.WEOOct2024`
  WHERE `WEO Subject Code` IN ('PPPGDP')
  AND Country IN (SELECT DISTINCT Country FROM `zkv-work-with-cc-pubdatasets.HighEd_Market_Indicators.annual_tuition_fees`)
  ),
gdp_long AS(
  SELECT Country, Year, GDP
  FROM gdp_data,
  UNNEST([
    STRUCT(2017 AS Year, `2017` AS GDP),
    STRUCT(2019 AS Year, `2019` AS GDP),
    STRUCT(2022 AS Year, `2022` AS GDP)
  ]) AS gdp_plus
  ),

-- Temp table 2 for GDPpc
gdppc_data AS(
  SELECT *
  FROM `zkv-work-with-cc-pubdatasets.HighEd_Market_Indicators.WEOOct2024`
  WHERE `WEO Subject Code` IN ('PPPPC')
  AND Country IN (SELECT DISTINCT Country FROM `zkv-work-with-cc-pubdatasets.HighEd_Market_Indicators.annual_tuition_fees`)
  ),
gdppc_long AS(
  SELECT Country, Year, GDPpc
  FROM gdppc_data,
  UNNEST([
    STRUCT(2017 AS Year, `2017` AS GDPpc),
    STRUCT(2019 AS Year, `2019` AS GDPpc),
    STRUCT(2022 AS Year, `2022` AS GDPpc)
  ]) AS gdppc_plus
  ),

-- Temp table 3 for Investment
investment_data AS(
  SELECT *
  FROM `zkv-work-with-cc-pubdatasets.HighEd_Market_Indicators.WEOOct2024`
  WHERE `WEO Subject Code` IN ('NID_NGDP')
  AND Country IN (SELECT DISTINCT Country FROM `zkv-work-with-cc-pubdatasets.HighEd_Market_Indicators.annual_tuition_fees`)
  ),
investment_long AS(
  SELECT Country, Year, Investment
  FROM investment_data,
  UNNEST([
    STRUCT(2017 AS Year, `2017` AS Investment),
    STRUCT(2019 AS Year, `2019` AS Investment),
    STRUCT(2022 AS Year, `2022` AS Investment)
  ]) AS investment_plus
  ),

-- Temp table 4 for Unemployment
unemployment_data AS(
  SELECT *
  FROM `zkv-work-with-cc-pubdatasets.HighEd_Market_Indicators.WEOOct2024`
  WHERE `WEO Subject Code` IN ('LUR')
  AND Country IN (SELECT DISTINCT Country FROM `zkv-work-with-cc-pubdatasets.HighEd_Market_Indicators.annual_tuition_fees`)
  ),
unemployment_long AS(
  SELECT Country, Year, Unemployment
  FROM unemployment_data,
  UNNEST([
    STRUCT(2017 AS Year, `2017` AS Unemployment),
    STRUCT(2019 AS Year, `2019` AS Unemployment),
    STRUCT(2022 AS Year, `2022` AS Unemployment)
  ]) AS unemployment_plus
  )

-- Joining all the tables
SELECT
  gdp_long.Country,
  gdp_long.Year,
  gdp_long.GDP,
  gdppc_long.GDPpc,
  investment_long.Investment,
  unemployment_long.Unemployment
FROM
  gdp_long
LEFT JOIN
  gdppc_long
  ON gdp_long.Country = gdppc_long.Country AND gdp_long.Year = gdppc_long.Year
LEFT JOIN
  investment_long
  ON gdp_long.Country = investment_long.Country AND gdp_long.Year = investment_long.Year
LEFT JOIN
  unemployment_long
  ON gdp_long.Country = unemployment_long.Country AND gdp_long.Year = unemployment_long.Year

"""

client.query(query_long_weo)